In [1]:
from pathlib import Path
import os
import sys

from dotenv import load_dotenv


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
    raise FileNotFoundError("Could not find repo root with app/ directory")


ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)
if os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"].strip()

print("ROOT_DIR:", ROOT_DIR)
print("KEY LOADED:", bool(os.getenv("OPENAI_API_KEY")))


ROOT_DIR: /Users/user/Desktop/RAG/backend
KEY LOADED: True


In [2]:
from app.rag.pipeline import RAGConfig, run_rag
import json

normalize_keywords = True  # 표준화 키워드 사용 (라우터 매칭 기반)

query = input("질문 입력: ").strip()
if not query:
    raise ValueError("질문을 입력해 주세요.")

config = RAGConfig(top_k=4, normalize_keywords=normalize_keywords)
result = await run_rag(query, config=config)

print("던진 질문:", query)
print("route:", result.get("routing", {}).get("route"))
print(json.dumps({
    "currentSituation": result.get("currentSituation", []),
    "nextStep": result.get("nextStep", []),
    "guidanceScript": result.get("guidanceScript", "")
}, ensure_ascii=False, indent=2))


던진 질문: 나라사랑카드 재발급
route: card_usage
{
  "currentSituation": [
    {
      "id": "card_33",
      "title": "나라사랑카드 재발급 신청 후 수령까지 얼마나 걸리나요?",
      "keywords": [
        "#나라사랑카드",
        "#재발급",
        "#수령기간"
      ],
      "content": "나라사랑카드 재발급은 신청일로부터 약 1주일이 소요됩니다. 입대 시 미소지자는 현장 재발급도 가능합니다.",
      "systemPath": "",
      "requiredChecks": [
        "재발급 신청일 확인",
        "입대 예정일 확인",
        "입영통지서 및 신분증 지참 여부"
      ],
      "exceptions": [
        "입대 예정일 2주 이내인 경우 현장 재발급 가능"
      ],
      "regulation": "",
      "detailContent": "제목: 나라사랑카드 재발급 신청 후 수령까지 얼마나 걸리나요?\n\n내용: 나라사랑카드는 일반 체크카드 발급기간보두 보통 3~4일 정도 더 소요되어, 재발급 신청일로부터 약 1주일이 소요됩니다. 그 이유는 의무자의 전자병역증(또는 전역증)이 탑재되는 시간이 소요되기 때문입니다. 입대 시 나라사랑카드를 미소지(분실, 도난 등)인 경우는 육군훈련소, 해군교육사, 공군교육사, 해병대교육훈련단 등에서는 현장 재발급이 가능하오니, 입대 예정일이 2주 이내인 대상자는 미소지한 상태로 입대하여도 무방합니다. (단, 입대 시에는 반드시 입영통지서와 신분증을 지참바랍니다.)",
      "time": "약 1주일",
      "note": "전자병역증(또는 전역증) 탑재로 인해 일반 체크카드보다 3~4일 더 소요됨",
      "relevanceScore": 0.015873015873015872
    },
   